# Vectorisation

One of the promises of JAX is to make vectorisation great again via the use of syntactic sugar decorators that describe what inputs are batched onto what outputs. The goal of this notebook is to show how this can be done in practice as well as how this is translated in terms of low-level code. 

## JAX imports

## Beginner
### Prerequisites
NumPy - (some exposure to Numba is helpful)

### Imports

In [1]:
import inspect

from jax import vmap, make_jaxpr
import jax.numpy as jnp

import numpy as np

### Example

To compare the vectorisation implementation of JAX to the NumPy one, let's take the following example:

In [2]:
def indexing_function(x, y):
    # Here x is a vector of floats, and y is a vector of ints
    return x[y]

We will use the following array for our tests:

In [3]:
N = 10

In [4]:
indexing_function(np.random.randn(N), np.random.randint(N))

np.float64(0.2560215528252095)

How does it react to batched inputs?

In [5]:
B = 3

In [6]:
indexing_function(np.random.randn(B, N), np.random.randint(N, size=B))

IndexError: index 3 is out of bounds for axis 0 with size 3

In [61]:
indexing_function(np.random.randn(B, N), np.random.randint(N))

IndexError: index 7 is out of bounds for axis 0 with size 3

OK so we need to modify it.

In [7]:
def complicated_indexing_function(x, y):
    # Here x is a vector of floats, and y is a vector of ints
    return x[..., y]

In [8]:
complicated_indexing_function(np.random.randn(B, N), np.random.randint(N))

array([-0.43011107,  0.41777996, -0.32865715])

In [9]:
complicated_indexing_function(np.random.randn(B, N), np.random.randint(N, size=B))

array([[-3.22292978, -1.91993084,  1.95687834],
       [ 0.28281618,  0.83102655, -1.19574312],
       [-1.52193677, -0.5091446 ,  0.84174498]])

Really not what we want!

### NumPy-style vectorisation

Instead of trying to be smart, let's use NumPy:

In [10]:
np_vectorised_indexing_function = np.vectorize(
    indexing_function, signature="(n),()->()"
)

In [11]:
np_vectorised_indexing_function(np.random.randn(B, N), np.random.randint(N))

array([-0.17239075, -1.62978962, -3.1590108 ])

And the JAX vectorisation:

In [12]:
jax_vectorised_indexing_function = jnp.vectorize(
    indexing_function, signature="(n),()->()"
)

In [13]:
jax_vectorised_indexing_function(np.random.randn(B, N), np.random.randint(N))

An NVIDIA GPU may be present on this machine, but a CUDA-enabled jaxlib is not installed. Falling back to cpu.


Array([0.7224782, 0.1067703, 1.4844942], dtype=float32)

So what is the difference?

In [14]:
batch_input = np.random.randn(10000, N)
batch_index = np.random.randint(N, size=10000)

In [15]:
%timeit np_vectorised_indexing_function(batch_input, batch_index)

8.03 ms ± 17.6 μs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [16]:
jax_batch_input = jnp.asarray(batch_input)
jax_batch_index = jnp.asarray(batch_index)

In [17]:
%timeit jax_vectorised_indexing_function(jax_batch_input, jax_batch_index).block_until_ready()

1.05 ms ± 8.27 μs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


Why is it faster? Because it's multi-threaded in the background!

### Vectorised map

On the other hand one can pick vmap: `jnp.vectorize` is a wrapper around the vmap functionality, this is useful in the case when the batching dimension is not the first one for example.

In [20]:
vmapped_indexing = vmap(indexing_function, in_axes=(1, 0))
# here we are saying that the input will be batched along
# the second dimension for the input, and the first for the index, this helps with not having to do shape arithmetics.

In [21]:
vmapped_indexing(np.random.randn(N, 3), np.random.randint(N, size=3))

Array([0.06808577, 1.1108743 , 0.9083771 ], dtype=float32)

### Questions:

#### Q1: 
Reimplement this manually vectorised function using `vmap`, and compare the generated code using `make_jaxpr`

In [22]:
def just_a_function(x, y):
    a = x[..., 0] * y[..., 1]
    b = x[..., 1] * y[..., 0]
    return a + b

In [23]:
just_a_function(np.random.randn(4, 3, 2), np.random.randn(4, 3, 2))

array([[ 0.15026824,  0.49542497,  0.8915288 ],
       [-0.24754084,  1.00554295,  1.13674269],
       [ 2.60848693,  0.1825349 , -1.07862578],
       [ 0.34280008, -2.13798014,  1.12652741]])

#### Q2:
Using `jnp.vectorize`, vectorise the following function with respect to the matrix `a`:
```python
def solve(a, b):
    return jnp.solve(a, b)
```

## Intermediate / Advanced
### Prerequisites
- Beginner vectorisation
- Beginner automatic differentiation
- Beginner loops (Advanced)

Now that we know how to vectorise, let's give an example where it's not only just a convenient wrapper but also a useful computational tool: we will see how to vectorise the JVP call we learned about in the automatic differentiation notebook.

### Imports

In [24]:
from functools import partial

from jax import make_jaxpr, jvp, vmap
import jax.numpy as jnp
from jax.random import normal, PRNGKey

import numpy as np

### Example

Let's take a simple example:

In [25]:
def fun(x):
    return jnp.sin(jnp.sum(x))

Say we want to compute its JVP against a number of random vectors:

In [26]:
def jvp_fun(x, key, d=100):
    n = x.shape[0]
    vectors = normal(key, shape=(n, d))
    return jvp(fun, (x,), (vectors,))

In [27]:
jvp_fun(jnp.array([0.0, 1.0]), PRNGKey(42))

ValueError: jvp called with different primal and tangent shapes;Got primal shape (2,) and tangent shape as (2, 100)

It doesn't work out of the box it seems... Let's try and obey the syntax of JVP:

In [28]:
def jvp_fun(x, key, d=20):
    n = x.shape[0]
    vectors = normal(key, shape=(n, d))
    return jvp(fun, (jnp.repeat(x.reshape(-1, 1), d, 1),), (vectors,))[1]

In [29]:
jvp_fun(jnp.array([0.0, 1.0]), PRNGKey(42))

Array(4.1461916, dtype=float32)

OK it's working so what's the problem here? `fun` is being relinearised at the same point $d$ times for no reason!

You can execute the line below to see this

In [30]:
# make_jaxpr(jvp_fun)(jnp.array([0., 1.]), PRNGKey(42))

We can actually solve this problem by using `vmap`:

In [31]:
def vmap_jvp_fun(x, key, d=20):
    n = x.shape[0]
    vectors = normal(key, shape=(n, d))
    local_fun = lambda vec: jvp(fun, (x,), (vec,))[1]
    return vmap(local_fun, in_axes=(1,))(vectors)

In [32]:
vmap_jvp_fun(jnp.array([0.0, 1.0]), PRNGKey(42))

Array([ 0.67665976,  0.29954007, -1.0683215 , -0.22646905,  0.98523813,
        1.1727225 ,  0.40628698,  0.43852165,  0.5092319 , -0.10380521,
        1.348889  , -0.7610773 ,  0.14572972,  0.07581756,  0.70275015,
        1.2393789 , -0.09658202, -1.0514277 ,  0.57944995,  0.21703981],      dtype=float32)

Execute the following line to compare with the naive manual approach

In [33]:
# make_jaxpr(vmap_jvp_fun)(jnp.array([0., 1.]), PRNGKey(42))

### Questions:

#### Q1:
Vectorise the following bubble sort algorithm using the method of your choice:
```python
def bubble_sort(arr): 
    n = len(arr) 
    res = np.copy(arr)
    for i in range(n-1): 
        for j in range(0, n-i-1): 
            if res[j] > res[j+1]: 
                res[j], res[j+1] = res[j+1], res[j]
    return res   
```